In [1]:
import json
import os
import sys
import math
import datetime
import pathlib

dir_path = pathlib.Path(os.path.abspath('')).resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat/OpenFormat-Public'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *

/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/encoding_size


In [4]:
num_rows = 1000 * 1000
num_cols = 20
sort_list = [
                0.01,
                0.2,
                0.4,
                0.6,
                0.8,
                1.0,
                1.2,
                1.4,
                1.6,
                1.8,
                2.0
            ]
type_list = ['int', 'float', 'string']

In [5]:
%%time
# generate csv files
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
!pwd
for t in type_list:
    print(t)
    for s in sort_list:
        print(s)
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        if os.path.exists(f'{dir_path}/{base_name}'):
            continue
        os.system(f'python3 gen_workloads.py \
            {t} {num_rows} {num_cols} {base_name} zipf {s}')
        os.system(f'cp -r {base_name} {dir_path} && \
            rm -r {base_name}')
os.chdir(dir_path)
!pwd


/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/benchmark/generator_v2
int
0.01
0.2
0.4
0.6
0.8
1.0
1.2
1.4
1.6
1.8
2.0
float
0.01
0.2
0.4
0.6
0.8
1.0
1.2
1.4
1.6
1.8
2.0
string
0.01
0.2
0.4
0.6
0.8
1.0
1.2
1.4
1.6
1.8
2.0
/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/encoding_size
CPU times: user 21.4 ms, sys: 8.63 ms, total: 30.1 ms
Wall time: 262 ms


In [6]:
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_encoding.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_encoding.json')
pq_name_suffix = '_' + "_".join([str(i) for i in list(pq_config[0].values())]) + '.parquet'
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
print(pq_config)

loading config from /home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/experiments/pq_encoding.json ...
loading config from /home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/experiments/orc_encoding.json ...
[{'compression': 'NONE', 'use_dictionary': True, 'dictionary_pagesize_limit': 2147483647, 'version': '2.6', 'data_page_version': '2.0'}]


In [18]:
# generate btrblocks schema
import yaml
os.chdir(f'{PROJ_SRC_DIR}/python')
for t in type_list:
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        arrow_schema_name = f'encoding_size/{base_name}/gen_data/{base_name}_arrow_schema.txt'
        with open(arrow_schema_name, 'r') as f:
            lines = f.readlines()
        columns = []
        cnt = 0
        for line in lines:
            if 'message' in line or '}' in line or len(line)==0:
                print(line)
                continue
            col_name = f'col{cnt}'
            col_type = None
            if 'String' in line:
                col_type = 'string'
            elif 'int' in line:
                col_type = 'integer'
            elif 'double' in line:
                col_type = 'double'
            if col_type is None:
                print(line)
                col_type = 'skip'
            cnt += 1
            columns.append({'name': col_name, 'type':col_type})
        with open(f'encoding_size/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml','w') as f:
            yaml.dump({'columns':columns}, f)

message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}


In [19]:
# generating files for btrblocks
os.chdir(f'{PROJ_SRC_DIR}/python')
for t in type_list:
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        os.system(f'mkdir encoding_size/{base_name}/gen_data/binary')
        os.system(f'mkdir encoding_size/{base_name}/gen_data/btr')
    
        os.system(f'~/btrblocks/build/csvtobtr -create_binary -binary encoding_size/{base_name}/gen_data/binary/ -csv encoding_size/{base_name}/gen_data/{base_name}.csv -yaml encoding_size/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml')
        os.system(f'~/btrblocks/build/csvtobtr -create_btr -btr encoding_size/{base_name}/gen_data/btr/ -binary encoding_size/{base_name}/gen_data/binary/ -csv encoding_size/{base_name}/gen_data/{base_name}.csv -yaml encoding_size/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml')

[2024-03-06 07:18:53.014] [info] Creating binary files in encoding_size/int_r1000000_c20_zipf0.01/gen_data/binary/
column name = col0 , type = integer
column name = col1 , type = integer
column name = col2 , type = integer
column name = col3 , type = integer
column name = col4 , type = integer
column name = col5 , type = integer
column name = col6 , type = integer
column name = col7 , type = integer
column name = col8 , type = integer
column name = col9 , type = integer
column name = col10 , type = integer
column name = col11 , type = integer
column name = col12 , type = integer
column name = col13 , type = integer
column name = col14 , type = integer
column name = col15 , type = integer
column name = col16 , type = integer
column name = col17 , type = integer
column name = col18 , type = integer
column name = col19 , type = integer
[haoyu.debug.Parser] writing binary fileencoding_size/int_r1000000_c20_zipf0.01/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/int_r1000

!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.


[2024-03-06 07:21:13.948] [info] Creating binary files in encoding_size/float_r1000000_c20_zipf0.2/gen_data/binary/
column name = col0 , type = double
column name = col1 , type = double
column name = col2 , type = double
column name = col3 , type = double
column name = col4 , type = double
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = double
column name = col15 , type = double
column name = col16 , type = double
column name = col17 , type = double
column name = col18 , type = double
column name = col19 , type = double
[haoyu.debug.Parser] writing binary fileencoding_size/float_r1000000_c20_zipf0.2/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/float_r1000000_c20_zipf0.2/

!!! compressed is larger than raw: 
for : exceptions - scheme = DICT difference = 9. Falling back to uncompressed.


[2024-03-06 07:21:55.544] [info] Creating binary files in encoding_size/float_r1000000_c20_zipf0.6/gen_data/binary/
column name = col0 , type = double
column name = col1 , type = double
column name = col2 , type = double
column name = col3 , type = double
column name = col4 , type = double
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = double
column name = col15 , type = double
column name = col16 , type = double
column name = col17 , type = double
column name = col18 , type = double
column name = col19 , type = double
[haoyu.debug.Parser] writing binary fileencoding_size/float_r1000000_c20_zipf0.6/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/float_r1000000_c20_zipf0.6/

!!! compressed is larger than raw: 
for : exceptions - scheme = DICT difference = 57. Falling back to uncompressed.


[2024-03-06 07:22:17.386] [info] Creating binary files in encoding_size/float_r1000000_c20_zipf0.8/gen_data/binary/
column name = col0 , type = double
column name = col1 , type = double
column name = col2 , type = double
column name = col3 , type = double
column name = col4 , type = double
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = double
column name = col15 , type = double
column name = col16 , type = double
column name = col17 , type = double
column name = col18 , type = double
column name = col19 , type = double
[haoyu.debug.Parser] writing binary fileencoding_size/float_r1000000_c20_zipf0.8/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/float_r1000000_c20_zipf0.8/

!!! compressed is larger than raw: 
for : values - scheme = DICT difference = 25. Falling back to uncompressed.


[2024-03-06 07:22:39.061] [info] Creating binary files in encoding_size/float_r1000000_c20_zipf1.0/gen_data/binary/
column name = col0 , type = double
column name = col1 , type = double
column name = col2 , type = double
column name = col3 , type = double
column name = col4 , type = double
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = double
column name = col15 , type = double
column name = col16 , type = double
column name = col17 , type = double
column name = col18 , type = double
column name = col19 , type = double
[haoyu.debug.Parser] writing binary fileencoding_size/float_r1000000_c20_zipf1.0/gen_data/binary/1_col0.bitmap
Writing binary file : encoding_size/float_r1000000_c20_zipf1.0/

In [20]:
# generate parquet and orc files
my_password = ''
os.chdir(f'{PROJ_SRC_DIR}/python')
for t in type_list:
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        os.system(f'cp {dir_path}/{base_name}/gen_data/{base_name}.csv data_gen')
        # os.system(f'echo {my_password} | sudo -S /home/haoyuq/miniconda3/envs/gen/bin/python scripts/parquet_exp.py {base_name} pq_encoding -r=false -o=false')
        os.system(f'echo {my_password} | sudo -S /home/haoyuq/miniconda3/envs/gen/bin/python scripts/orc_exp.py {base_name} orc_encoding -r=false -o=false')
        # orc_schema = open(f'encoding_size/{base_name}/gen_data/{base_name}_orc_schema.txt', 'r').read()
        # os.system(f'{HOME_DIR}/orc/orc/build/tools/src/csv-import \"{orc_schema}\" \
        #         encoding/{base_name}/gen_data/{base_name}.csv {base_name}.orc')
        # os.system(f'mv {base_name}{pq_name_suffix} {dir_path}/{base_name}.parquet')
        os.system(f'mv {base_name}{orc_name_suffix} {dir_path}/{base_name}.orc')
os.chdir(dir_path)

[sudo] password for haoyuq: 

loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 2.578284978866577


[sudo] password for haoyuq: 

loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 2.556161642074585


[sudo] password for haoyuq: 

...

loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 2.4977102279663086


[sudo] password for haoyuq: 

loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 2.2407820224761963


[sudo] password for haoyuq: 

loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 2.092606544494629


In [23]:
# collect size
def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total
    
os.chdir(dir_path)
os.system('rm outputs/stats.json')
for t in type_list:
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        output_stats = {}
        output_stats['type'] = t
        output_stats['feature'] = 'zipf'
        output_stats['param'] = s
        output_stats['file'] = 'parquet'
        output_stats['size'] = os.path.getsize(f'{base_name}.parquet')
        parse_output(output_stats)
        output_stats['file'] = 'orc'
        output_stats['size'] = os.path.getsize(f'{base_name}.orc')
        parse_output(output_stats)
        output_stats['file'] = 'btrblocks'
        output_stats['size'] = get_dir_size(f'{base_name}/gen_data/btr')
        parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('encoding_size_zipf', timestamp))

0